The following code uses cross validation with 3 folds to determine a model with the best misclassification rate for 3 logistic loss functions with L1, L2, and no regularizers. Additionally, a hinge loss model with quadratic regularization is tested. The classifier is very unbalanced (Less than 1% of the data is for trips greater than 3600 seconds). All cross validation methods pick the test set with no trips greater than 3600 seconds so that the misclassification rate is easily 0. From this, we conclude that trips greater than 3600 seconds are outliers. 

By Allison Grimsted

In [1]:
using Pkg
using Random
Random.seed!(13)
using Dates
using LinearAlgebra, SparseArrays, LowRankModels
include("proxgrad.jl")
using CSV
using Plots
using DataFrames
using Statistics
using LinearAlgebra
using MLBase

In [2]:
df = CSV.read("BikeWeatherZillow.csv");

### Addition of Day of the Week

In [3]:
df[!, :week_day]=dayname.(DateTime.(2018, df[:, :start_month], df[:, :start_day]));

### Deletion of Redundant Features

In [4]:
delete!(df, :start_day)
delete!(df, :start_min)
delete!(df, :start_station_latitude)
delete!(df, :start_station_longitude)
delete!(df, :start_station_zipcode);

┌ Warning: `delete!(df::AbstractDataFrame, cols::Any)` is deprecated, use `select!(df, Not(cols))` instead.
│   caller = top-level scope at In[4]:1
└ @ Core In[4]:1
┌ Warning: `delete!(df::AbstractDataFrame, cols::Any)` is deprecated, use `select!(df, Not(cols))` instead.
│   caller = top-level scope at In[4]:2
└ @ Core In[4]:2
┌ Warning: `delete!(df::AbstractDataFrame, cols::Any)` is deprecated, use `select!(df, Not(cols))` instead.
│   caller = top-level scope at In[4]:3
└ @ Core In[4]:3
┌ Warning: `delete!(df::AbstractDataFrame, cols::Any)` is deprecated, use `select!(df, Not(cols))` instead.
│   caller = top-level scope at In[4]:4
└ @ Core In[4]:4
┌ Warning: `delete!(df::AbstractDataFrame, cols::Any)` is deprecated, use `select!(df, Not(cols))` instead.
│   caller = top-level scope at In[4]:5
└ @ Core In[4]:5


We decided to now use longitude, latitude, or zipcode because it confounds with start_station_id, which is more granular than the other 2 geographic choices. 

### Casting of Types

In [5]:
df[!, :start_month]=string.(df[!, :start_month])
df[!, :start_station_id]=string.(df[!, :start_station_id])
df[!, :gender]=string.(df[!, :gender]);

In [6]:
feature_names = names(df)
for i in 1:length(feature_names)
    println(string(i), "\t", string(feature_names[i]), "\t\t\t", string(eltype(df[!, i])))
end

1	trip_duration			Int64
2	start_month			String
3	start_hour			Int64
4	start_station_id			String
5	user_type			String
6	birth_year			Int64
7	gender			String
8	total_precipitation_inches			Float64
9	average_temperature_farenheit			Float64
10	total_snowfall_inches			Float64
11	median_rental_price			String
12	median_sale_price			String
13	week_day			String


# Train/Test Split

In [7]:
df = df[shuffle(1:end), :] # we shuffle the data so that our train/test split will be truly random

println("Size of dataset: ", string(n))

#response variable
target = df[:, :trip_duration] 
#explanatory variables
data = df[:, filter(col -> (col != :trip_duration), feature_names)]

Size of dataset: 597780


,start_month,start_hour,start_station_id,user_type,birth_year,gender,total_precipitation_inches
,String,Int64,String,String,Int64,String,Float64
1,2,16,3002,"""Subscriber""",1964,2,0.67
2,1,16,519,"""Subscriber""",1979,1,0.05
3,6,16,3472,"""Subscriber""",1990,1,0.0
4,1,15,164,"""Subscriber""",1969,1,0.0
5,4,7,3108,"""Subscriber""",1970,1,0.02
6,2,7,3357,"""Subscriber""",1966,1,0.68
7,6,17,359,"""Subscriber""",1970,1,0.02
8,3,11,3140,"""Subscriber""",1979,1,0.0
9,2,21,477,"""Subscriber""",1964,1,0.0


# Turning the Data into a Matrix with One Hot Encoding:

In [15]:
"Computes a onehot vector for every entry in column given a set of categories cats"
function onehot(column, cats=unique(column))
    result = zeros(size(column, 1),size(cats, 1))
    for i in 1:size(column, 1)
        for j in 1:(size(cats, 1))
            if(column[i]==cats[j])
                result[i,j]=1  
            end
        end
    end
    return result
end;

In [14]:
real_labs = [
    :start_hour,
    :birth_year,
    :total_precipitation_inches,
    :average_temperature_farenheit,
    :total_snowfall_inches
]
real = data[:, filter(x -> (x in real_labs), names(df))]
xReal = convert(Matrix, hcat(real, ones(length(real[:, 1]))))
cat_labels = [
    :start_month,
    :start_station_id,
    :user_type,
    :gender,
    :week_day
]

cats_sets = [unique(data[:, label]) for label in cat_labels]
xCat = hcat([onehot(data[:, cat_labels[i]], cats_sets[i]) for i in 1:size(cat_labels, 1)]...)
X = hcat(convert(Matrix, xCat), xReal)

597780×841 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  …  16.0  1964.0  0.67  42.8  0.0   1.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0     16.0  1979.0  0.05  30.5  0.43  1.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0     16.0  1990.0  0.0   73.8  0.0   1.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0     15.0  1969.0  0.0   33.7  0.0   1.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0      7.0  1970.0  0.02  46.7  0.0   1.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  …   7.0  1966.0  0.68  32.5  2.94  1.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0     17.0  1970.0  0.02  69.9  0.0   1.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0     11.0  1979.0  0.0   35.5  0.0   1.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0     21.0  1964.0  0.0   31.6  0.0   1.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0     18.0  1988.0  0.0   62.9  0.0   1.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  …  16.0  1964.0  0.0   44.8  0.0   1.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0     13.0  1955.0  0.02  77.7  0.0   1.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0     18.0  1976.0  0.0   69.7  0.0   

# Function to Create a Binary Vector

In [16]:
#determines if each entry is above or below the cutoff in a vector 
function toBool(y, cutoff)
    for i in 1:length(y)
        if y[i]<= cutoff
            y[i]=-1
        else
            y[i]=1
        end
    end
    return y
end

toBool (generic function with 1 method)

## Logistic Loss

In [30]:
#y MUST be a one dimensional vector
function logReg(x, y)
    yNew = toBool(y, 3600)
    n = length(yNew)
    loss = 1/n * LogisticLoss()
    lambda = 0.5
    reg = lambda * QuadReg()
    w = proxgrad(loss, reg, x, yNew, maxiters=1000)
    return w
end

logReg (generic function with 1 method)

## Logistic Loss with no regularization

In [26]:
#Logistic Loss with no regularization
function logReg_noreg(x, y)
    yNew = toBool(y, 3600)
    n = length(yNew)
    loss = 1/n * LogisticLoss()
    reg = ZeroReg()
    w = proxgrad(loss, reg, x, yNew, maxiters=1000)
    return w
end

logReg_noreg (generic function with 1 method)

## Logistics Loss with L1 regularization

In [32]:
function log_l1(x, y)
    yNew = toBool(y, 3600)
    n = length(yNew)
    loss = 1/n * LogisticLoss()
    reg =  OneReg()
    w = proxgrad(loss, reg, x, yNew, maxiters=100)
    return w
end

log_l1 (generic function with 1 method)

## Hinge Loss

In [34]:
function hingeLoss(x, y)
    yNew = toBool(y, 3600)
    n = length(yNew)
    loss = 1/n * HingeLoss()
    lambda = 0.5
    reg = lambda * QuadReg()
    w = proxgrad(loss, reg, x, yNew, maxiters=1000)
    return w
end

hingeLoss (generic function with 1 method)

# Cross Validation 

In [45]:
function crossValidate(x, y, n)
    train_ind = collect(Kfold(length(y), n))
    len = length(y)
    wid = length(x[1,:])
    wBest = zeros(wid)
    currErr = 60*60+1
    bestErr = currErr
    wCurr = zeros(wid)
    for i in 1:n
        test_ind = setdiff(1:n, train_ind[i])
        trainX = x[train_ind[i], :]
        testX = x[test_ind, :]
        trainY = y[train_ind[i]]
        testY = y[test_ind]
        wCurr = hingeLoss(trainX, trainY)               #Change this line to desired function
        currErr = misclassification(testX, testY, wCurr)
        if currErr < bestErr
            bestErr = currErr
            wBest = wCurr
        end
    end
    return wBest, bestErr
end

crossValidate (generic function with 1 method)

## Misclassification Rate

In [29]:
# calculates the misclassification rate of a model
function misclassification(X, y, w)
    n = size(X,1)
    origY = toBool(y, 3600)
    newY = toBool(y, 0)
    mis = 0
    for i in 1:n
        if(origY[i] != newY[i])
            mis += 1/n
        end
    end
    return round(mis, digits=4)
end

misclassification1 (generic function with 1 method)

## Function Calls

In [22]:
w_logReg, error_logReg = crossValidate(X,target, 3)

([-1.042716996239396e-5, -1.588127003614895e-5, 1.0421272785230963e-5, 7.653004851365726e-6, -1.5356332816790094e-6, -8.338129565805937e-6, 9.155730402559746e-7, -9.629875777393462e-6, 5.658150421276018e-6, 1.0700888691295305e-5  …  -1.490864084278504e-5, -7.555384582258645e-6, -6.186338250347737e-6, -1.4561564497463593e-5, -8.128240416746967e-5, -0.003016622451903343, -1.3823152883294088e-6, 0.001479113277729779, -2.7515214389139078e-6, -1.1406895608615365e-6], 0.0)

In [28]:
w_logReg_noreg, error_logReg_noreg = crossValidate(X,target, 3)

([-9.766135227305574e-6, -1.6790114180676412e-5, 1.1344333601960038e-5, 8.165087461190189e-6, -2.567915209265128e-6, -1.0625449424391978e-5, 8.901429453794082e-7, -8.00293652630193e-6, 3.2976785095220045e-6, 9.019845322698985e-6  …  -1.3694303450085112e-5, -8.557017579974919e-6, -4.650122509993374e-6, -1.2599154653854922e-5, -5.288189448437128e-5, -0.0030288903165597517, 1.798213041267922e-7, 0.0014526068733458988, -2.805960132377823e-6, -1.171633379395286e-6], 0.0)

In [43]:
w_log_l1, error_log_l1 = crossValidate(X,target, 3)

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, -0.002370543990590287, 0.0, 0.0, 0.0, 0.0], 0.0)

In [48]:
w_hingeLoss, error_hingeLoss = crossValidate(X,target, 3)

([-6.386041084038622e-7, -9.328262202489706e-7, 5.987350942871378e-7, 2.853874102059842e-7, -4.864225046413742e-7, -6.321441179517944e-8, 2.8216465679726344e-7, -4.502825383846242e-7, 3.358756218864266e-7, 4.284928968253213e-7  …  -8.48886496714518e-7, -4.144879440334566e-7, -2.85900543805101e-7, -7.402645744924923e-7, -2.3845823543902846e-6, -0.0005152796742550165, -1.9500162569416545e-7, 7.657022992474948e-5, -1.7946716861046202e-7, -2.4666547181012736e-7], 0.0)